<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dependencies" data-toc-modified-id="Dependencies-1">Dependencies</a></span></li><li><span><a href="#Functions" data-toc-modified-id="Functions-2">Functions</a></span></li><li><span><a href="#Paths" data-toc-modified-id="Paths-3">Paths</a></span></li><li><span><a href="#Main" data-toc-modified-id="Main-4">Main</a></span></li><li><span><a href="#NMI" data-toc-modified-id="NMI-5">NMI</a></span></li></ul></div>

# Dependencies

In [1]:
pacman::p_load(igraph, graphlayouts, ggraph, ggforce, dplyr)

# Functions

In [2]:
#source("/Users/anabbi/git/ped_CapTCRseq/R/Immune_diversity.R")

# Paths

In [3]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"

gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

In [4]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/"

# Main

In [5]:
aaT <- read.csv(file = paste0(datapath, "aa_attributes.csv"), header = T)

In [6]:
head(aaT)

,Amino.acid,Factor.I,Factor.II,Factor.III,Factor.IV,Factor.V
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,A,−0.591,−1.302,−0.733,1.57,−0.146
2,C,−1.343,0.465,−0.862,−1.020,−0.255
3,D,1.05,0.302,−3.656,−0.259,−3.242
4,E,1.357,−1.453,1.477,0.113,−0.837
5,F,−1.006,−0.590,1.891,−0.397,0.412
6,G,−0.384,1.652,1.33,1.045,2.064


In [7]:
load(file = paste0(datapath, "capTCRseq/INT_allfiles.RData"))

In [8]:
head(intall)

,cloneId,cloneCount,cloneFraction,clonalSequence,clonalSequenceQuality,allVHitsWithScore,allDHitsWithScore,allJHitsWithScore,allCHitsWithScore,allVAlignments,⋯,aaSeqFR2,aaSeqCDR2,aaSeqFR3,aaSeqCDR3,aaSeqFR4,refPoints,TRBV,TRBJ,file,samplename
,<int>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<lgl>,<lgl>,<lgl>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0,187,0.19767442,TGTGCCAGCAGCTTCTATGGCAGGGAAGCAGATACGCAGTATTTT,ENNNNNENNEENENENENNNEENNNEENNNNEKNNNNNNENNNNN,TRBV7-2*00(367.3),TRBD1*00(25),TRBJ2-3*00(224.8),NA,580|594|617|0|14||70.0,⋯,NA,NA,NA,CASSFYGREADTQYF,NA,:::::::::0:-3:14:20:-4:-3:25:28:-4:45:::,TRBV7-2,TRBJ2-3,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315-05-PBMC-DNA
2,3,97,0.10253700,TGTGCCAGCAGTTACTACAAAGCGGCAGGGGGAAGAGACCCAGTACTTC,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE,"TRBV6-2*00(391.9),TRBV6-3*00(391.9)","TRBD1*00(35),TRBD2*00(34)",TRBJ2-5*00(224),NA,516|532|553|0|16||80.0;516|532|553|0|16||80.0,⋯,NA,NA,NA,CASSYYKA_QGEETQYF,NA,:::::::::0:-1:16:25:-4:-1:32:32:-3:49:::,TRBV6-2,TRBJ2-5,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315-05-PBMC-DNA
3,14,48,0.05073996,TGTGCCAGCAGCTTAATGGGGGGAGATAGCACTGAAGCTTTCTTT,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE,TRBV7-3*00(159.7),"TRBD1*00(25),TRBD2*00(25)",TRBJ1-1*00(220),NA,540|556|577|0|16||80.0,⋯,NA,NA,NA,CASSLMGGDSTEAFF,NA,:::::::::0:-1:16:17:-6:-1:22:29:-4:45:::,TRBV7-3,TRBJ1-1,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315-05-PBMC-DNA
4,15,48,0.05073996,TGTGCCAGCAGCTTAACCGGGACAGGGGTTGAAAAACTGTTTTTT,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE,TRBV7-3*00(765.2),TRBD1*00(55),TRBJ1-4*00(145),NA,540|557|577|0|17||85.0,⋯,NA,NA,NA,CASSLTGTGVEKLFF,NA,:::::::::0:0:17:17:1:-2:28:29:-7:45:::,TRBV7-3,TRBJ1-4,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315-05-PBMC-DNA
5,20,42,0.04439746,TGTGCCAGCAGCCAACGGGGACAGTCAACCGAGCAGTACTTC,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE,TRBV14*00(519.2),TRBD1*00(35),TRBJ2-7*00(205),NA,516|531|553|0|15||75.0,⋯,NA,NA,NA,CASSQRGQSTEQYF,NA,:::::::::0:-2:15:17:0:-5:24:29:-6:42:::,TRBV14,TRBJ2-7,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315-05-PBMC-DNA
6,29,32,0.03382664,TGTGCCAGCAGTTACCAAGCGGGGGTGGGTGAGACCCAGTACTTC,EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE,TRBV6-6*00(344),TRBD2*00(30),TRBJ2-5*00(214),NA,516|531|553|0|15||75.0,⋯,NA,NA,NA,CASSYQAGVGETQYF,NA,:::::::::0:-2:15:17:-6:-4:23:30:-5:45:::,TRBV6-6,TRBJ2-5,/Users/anabbi/Desktop/H4H/INTERCEPT/NextSeq/batch4_19Aug2019/mixcr/clones/CLONES_TRBCHP_315-05-PBMC-DNA.txt,CHP_315-05-PBMC-DNA


In [10]:
intall$aa_length <- nchar(intall$aaSeqCDR3)

In [11]:
summary(intall$aa_length)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   4.00   14.00   15.00   18.27   16.00  118.00 

In [13]:
intall_subset <- intall[ intall$aa_length == 15,]

In [16]:
dim(intall_subset)

[1] 20187    40

In [17]:
head(intall_subset$aaSeqCDR3)

[1] "CASSFYGREADTQYF" "CASSLMGGDSTEAFF" "CASSLTGTGVEKLFF" "CASSYQAGVGETQYF"
[5] "CASSVVGVYNSPLHF" "CASSLTRPSGNTIYF"

In [18]:
strsplit("CASSFYGREADTQYF", "")

[[1]]
 [1] "C" "A" "S" "S" "F" "Y" "G" "R" "E" "A" "D" "T" "Q" "Y" "F"

# NMI

In [28]:
mat <- sapply(as.character(c("CASSFYGREADTQYF", "CASSVVGVYNSPLHF", "CASSLTRPSGNTIYF",
                            "CASSYQAGVGETQYF")), function(x){
      x <- as.character(x);
      x <- strsplit(x, "");
      x <- unlist( x )
   })

mat <- t(mat)

In [29]:
 res <- data.frame()
   for( i in 1:15 ){
   for( k in 1:15 ){
      print(paste0( i, ":", k))
      d1 <- mat[, c(i,k)]
      d1 <- as.data.frame(d1)
      colnames(d1) <- c("X","Y")
      d1 <- d1[ d1$X!="NA" &  d1$Y!="NA",] #revemo NA
      
      xlist <- as.character( unique(d1$X) )
      ylist <- as.character( unique(d1$Y) )
      
      #make frequency table
      freq <- matrix(0, nrow=length(xlist), ncol=length(ylist))
      row.names(freq) <- xlist
      colnames(freq) <- ylist
      
      for( x in xlist ){
      for( y in ylist ){
         both <- subset(d1, X==x & Y==y)
         freq[ x, y ] <- nrow( both )
      }
      }
      
      MI <- mi.empirical(freq, unit=c("log"))
      Hx <- entropy.empirical( colSums(freq) )
      Hy <- entropy.empirical( rowSums(freq) )
      NMI <- 2 * MI / ( Hx + Hy ) #normalized mutual entropy
      
      dump <- data.frame(i, k, MI, Hx, Hy, NMI)
      res <- rbind(res, dump)
      
   }
   }

[1] "1:1"
[1] "1:2"
[1] "1:3"
[1] "1:4"
[1] "1:5"
[1] "1:6"
[1] "1:7"
[1] "1:8"
[1] "1:9"
[1] "1:10"
[1] "1:11"
[1] "1:12"
[1] "1:13"
[1] "1:14"
[1] "1:15"
[1] "2:1"
[1] "2:2"
[1] "2:3"
[1] "2:4"
[1] "2:5"
[1] "2:6"
[1] "2:7"
[1] "2:8"
[1] "2:9"
[1] "2:10"
[1] "2:11"
[1] "2:12"
[1] "2:13"
[1] "2:14"
[1] "2:15"
[1] "3:1"
[1] "3:2"
[1] "3:3"
[1] "3:4"
[1] "3:5"
[1] "3:6"
[1] "3:7"
[1] "3:8"
[1] "3:9"
[1] "3:10"
[1] "3:11"
[1] "3:12"
[1] "3:13"
[1] "3:14"
[1] "3:15"
[1] "4:1"
[1] "4:2"
[1] "4:3"
[1] "4:4"
[1] "4:5"
[1] "4:6"
[1] "4:7"
[1] "4:8"
[1] "4:9"
[1] "4:10"
[1] "4:11"
[1] "4:12"
[1] "4:13"
[1] "4:14"
[1] "4:15"
[1] "5:1"
[1] "5:2"
[1] "5:3"
[1] "5:4"
[1] "5:5"
[1] "5:6"
[1] "5:7"
[1] "5:8"
[1] "5:9"
[1] "5:10"
[1] "5:11"
[1] "5:12"
[1] "5:13"
[1] "5:14"
[1] "5:15"
[1] "6:1"
[1] "6:2"
[1] "6:3"
[1] "6:4"
[1] "6:5"
[1] "6:6"
[1] "6:7"
[1] "6:8"
[1] "6:9"
[1] "6:10"
[1] "6:11"
[1] "6:12"
[1] "6:13"
[1] "6:14"
[1] "6:15"
[1] "7:1"
[1] "7:2"
[1] "7:3"
[1] "7:4"
[1] "7:5"
[1] "7:6"
[1] 

In [34]:
res

i,k,MI,Hx,Hy,NMI
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,0,0.0000000,0,NaN
1,2,0,0.0000000,0,NaN
1,3,0,0.0000000,0,NaN
1,4,0,0.0000000,0,NaN
1,5,0,1.3862944,0,0
1,6,0,1.3862944,0,0
1,7,0,1.0397208,0,0
1,8,0,1.3862944,0,0
1,9,0,1.3862944,0,0
